In [1]:
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate

# For caching models and logging
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 83.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 87.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 14.4 MB/s eta 0:00:0000:0

In [2]:
import torch

# Check GPU status
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Available Memory: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")

# Clear cache
torch.cuda.empty_cache()

GPU: Tesla T4
Total Memory: 15.83 GB
Available Memory: 15.83 GB


In [3]:
import os
os.environ['VLLM_USE_V1'] = '0'  # Disable V1 engine

# Clear any existing GPU memory
import torch
import gc

# Clear CUDA cache
torch.cuda.empty_cache()
gc.collect()

# Try to free all CUDA memory
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        with torch.cuda.device(i):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            
import pandas as pd
from vllm import LLM, SamplingParams
from tqdm import tqdm
import gc
import torch

# Load the CSV
df = pd.read_csv('/kaggle/input/banglachq/train.csv')
print(f"Loaded {len(df)} rows from train.csv")
print(f"Columns: {df.columns.tolist()}")

# Load model with vLLM on single GPU with optimized settings
model_name = "Qwen/Qwen2.5-3B-Instruct"
print(f"\nLoading {model_name} with vLLM...")

llm = LLM(
    model=model_name,
    trust_remote_code=True,
    dtype="float16",
    gpu_memory_utilization=0.65,  # Reduced slightly for larger context
    max_model_len=2048,  # Increased to 2048
    max_num_seqs=24,  # Reduced batch size slightly
)

# Get tokenizer for chat template
tokenizer = llm.get_tokenizer()

print("Model loaded successfully!\n")

# Translation function (prompt already in Chatgaiyan)
def translate_to_chatgaiyan(bangla_text):
    prompt = f"""You are a precise translation tool. Your task is to translate the given Bangla text to Chittagonian (Chatgaiyan) dialect using Bengali script.

        INSTRUCTIONS:
        - Analyze the text step-by-step before translating
        - Use only Bengali script in your final translation
        - After your reasoning, provide ONLY the final translation with no additional commentary

        Here are examples showing the translation process:

        Example 1:
        Bangla: খালু, গ্রামের বাড়ি যাবেন কবে
        Reasoning: "খালু" becomes "অহালু" (phonetic shift with অ prefix), "গ্রামের বাড়ি" becomes "গ্রামর বাড়িত" (possessive -এর→-র, locative case -ত added), "যাবেন কবে" becomes "হত্তে যাইবেন?" (emphatic হত্তে added, verb form changed, question marker)
        Chittagonian: অহালু, গ্রামর বাড়িত হত্তে যাইবেন?

        Example 2:
        Bangla: আপনি কি নিয়মিত জল খান?
        Reasoning: "আপনি" becomes "অনে" (phonetic compression), "কি নিয়মিত" becomes "কি পইত্তেন" (restructured with পইত্তেন for regularity concept), "জল খান" becomes "পানি হন না?" (জল→পানি, খান→হন না with negative question form)
        Chittagonian: অনে কি পইত্তেন পানি হন না?

        Example 3:
        Bangla: জুম্মার পর যাব।
        Reasoning: "জুম্মার" becomes "জুমর" (possessive -আর→-র shortened), "পর" stays as "পর" (unchanged), "যাব" becomes "যাইয়ুম" (first person future tense -ব→-ইয়ুম)
        Chittagonian: জুমর পর যাইয়ুম।

        Example 4:
        Bangla: পালং শাক খুব পছন্দ
        Reasoning: "পালং শাক" stays as "পালংশাক" (compound unchanged), "খুব" becomes "বেশি" (lexical replacement), "পছন্দ" becomes "গম লাগে দে" (idiomatic expression with গম=liking, লাগে=feels, দে=emphasis particle)
        Chittagonian: পালংশাক বেশি গম লাগে দে

        Example 5:
        Bangla: বাঙালি নানা-নানীরা কি করেন?
        Reasoning: "বাঙালি" becomes "বাআলর" (phonetic compression with possessive -র), "নানা-নানী-রা" becomes "নানানানী অলে" (compound simplified, plural -রা→অলে), "কি করেন" becomes "কি গরে দে?" (করেন→গরে with দে emphasis particle)
        Chittagonian: বাআলর নানানানী অলে কি গরে দে?

        Now translate this text:
        Bangla: {bangla_text}

        Think through the translation:
        - Identify key vocabulary transformations
        - Note grammatical changes (possessives, verb forms, cases)
        - Apply Chittagonian phonetic and structural patterns

        Provide your final translation below:
        Chittagonian:""" 

    messages = [{"role": "user", "content": prompt}]

    # Apply chat template
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    return formatted_prompt

# Set sampling parameters
sampling_params = SamplingParams(
    temperature=0.3,
    top_p=0.9,
    max_tokens=200
)

# Process in smaller batches to avoid OOM
BATCH_SIZE = 32  # Match max_num_seqs
all_translations = []

print(f"Processing {len(df)} sentences in batches of {BATCH_SIZE}...")

for batch_start in tqdm(range(0, len(df), BATCH_SIZE), desc="Processing batches"):
    batch_end = min(batch_start + BATCH_SIZE, len(df))
    batch_df = df.iloc[batch_start:batch_end]
    
    # Prepare prompts for this batch
    batch_prompts = []
    for idx, row in batch_df.iterrows():
        bangla_text = row['question']
        prompt = translate_to_chatgaiyan(bangla_text)
        batch_prompts.append(prompt)
    
    # Generate translations for this batch
    outputs = llm.generate(batch_prompts, sampling_params)
    
    # Extract translations
    batch_translations = [output.outputs[0].text.strip() for output in outputs]
    all_translations.extend(batch_translations)
    
    # Clear memory after each batch
    del outputs, batch_prompts
    gc.collect()
    torch.cuda.empty_cache()

# Print first few examples
print("\nFirst 3 examples:")
for idx in range(min(3, len(df))):
    print(f"\nBangla: {df.iloc[idx]['question']}")
    print(f"Chatgaiyan: {all_translations[idx]}")

# Add translations to dataframe
df['chatgaiyan_translation'] = all_translations

# Save results
df.to_csv('qwen-5-shot-chittagongian-CoT.csv', index=False)
print(f"\n✅ Translation complete! Saved to 'qwen_10_train_with_chatgaiyan_translations.csv'")
print(f"Total sentences translated: {len(df)}")


2025-09-29 13:29:23.503134: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759152563.742176      73 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759152563.807689      73 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 09-29 13:29:38 [__init__.py:216] Automatically detected platform cuda.
Loaded 1880 rows from train.csv
Columns: ['id', 'question', 'indices', 'summary']

Loading Qwen/Qwen2.5-3B-Instruct with vLLM...
INFO 09-29 13:29:40 [utils.py:328] non-default args: {'trust_remote_code': True, 'dtype': 'float16', 'max_model_len': 2048, 'gpu_memory_utilization': 0.65, 'max_num_seqs': 24, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-3B-Instruct'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

INFO 09-29 13:29:54 [__init__.py:742] Resolved architecture: Qwen2ForCausalLM


...

Processing batches:  98%|█████████▊| 58/59 [36:15<00:37, 37.82s/it]

Adding requests:   0%|          | 0/24 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/24 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processing batches: 100%|██████████| 59/59 [36:41<00:00, 37.31s/it]


First 3 examples:

Bangla: আমার বয়স ২৫ বছর । আজ কিছুদিন যাবত আমার ঘন ঘন টয়লেট হচ্ছে । হটাৎ পেটের মধ্যে ব্যাথা অনুভূত হয়ে টয়লেটে চাপ দেয় এবং টয়লেটে যাওয়ার পরে ও টয়লেট ক্লিয়ারলি হচ্ছে না । এই সমস্যাটার সমাধান জানাবেন । আর একটি সমস্যা আছে , সেটি হল আজ দুই দিন যাবত হটাৎ করে আমার ঠান্ডা লেগে সর্দি + অনেক হাচি হচ্ছে । 
Chatgaiyan: অনে বয়স ২৫ বছর । আজ কিছুদিন যাবত অনে ঘন ঘন টয়লেট হচ্ছে । হটাৎ পেটের মধ্যে ব্যাথা অনুভূত হয়ে টয়লেটে চাপ দেয় এবং টয়লেটে যাওয়ার পরে অনে টয়লেট ক্লিয়ারলি হচ্ছে না । এই সমস্যাটার সম

Bangla: বাচ্চার বয়স ১০ মাস চলছে , ওজন ৮ . ৩ কেজি । দাঁত উঠতেছে , হালকা ঘন সর্দি আজ ৪ দিন যাবত , হাছি কম । মায়ের বুকের দুধ ছাড়া অন্য কোন খাবার ১০ দিন ধরে খায় না , এমন কি পানিও কম খায় । সর্দি ভাল করার জন্য উপায় কী এবং খাবার খাওয়ানোর ব্যাপারে কী করব ? 
Chatgaiyan: বাআলর বাচ্চার অলে ১০ মাস চলছে, পানি ৮ . ৩ কেজি। দাঁত উঠছে, হালকা ঘন সর্দি আজ ৪ দিন যাবত, হাছি কম। মায়ের বুকের দুধ ছাড়া অন্য কোন খাবার ১০ দিন ধরে খায় না, এমন কি পানি কম খায়। সর্দি ভাল

Bangla: আমার সব সময় চুল পড়ে । প্রায়